In [1]:
from agents import function_tool
import wikipedia
import requests
import os 
from dotenv import load_dotenv
load_dotenv()
os.environ["OPENAI_API_KEY"] = os.environ['OPENAI_API_KEY_MALI']
from serpapi import GoogleSearch
api_key = os.getenv("SERPAPI_API_KEY")
import openmeteo_requests
from data_models.flight_data_models import FlightSearchResults, OtherFlight, BestFlight

import nest_asyncio
nest_asyncio.apply()

from agents.model_settings import ModelSettings
from agents import Agent, Runner,function_tool,trace, ItemHelpers, MessageOutputItem
from data_models.hotel_data_models import Property
from data_models.yelp_data_models import Category, Business
from helpers.helper_functions import assemble_conversation

from helpers.function_tools import flight_search_2, hotels_search2, yelp_search2
from helpers.travel_agents import t_agent,f_agent,h_agent,y_agent
from pydantic import BaseModel
from typing import Optional
from agents import Agent, handoff, RunContextWrapper
from data_models.first_question_output_format import FinalModel, YelpOutput, HotelOutput, FlightOutput
from agents.extensions.handoff_prompt import RECOMMENDED_PROMPT_PREFIX

END OF THE SCRIPT


In [2]:
def flight_to_string(response: dict, flight_key: str) -> str:
    """
    Converts flight information to a human-readable string.

    Args:
        response (dict): The response from the API.
        flight_key (str): Key to select 'best_flights' or 'other_flights'.

    Returns:
        str: A string summary of the flight data.
    """
    text_summary = ""

    for i, flight_data in enumerate(response.get(flight_key, [])):
        flight = OtherFlight(**flight_data)

        text_summary += f"\n***** Flight {i + 1} *****\n"
        text_summary += (
            f"Flight Summary:\n"
            f"Type: {flight.type}\n"
            f"Total Duration: {flight.total_duration} minutes\n"
            f"Price: ${flight.price}\n"
        )

        for j, segment in enumerate(flight.flights):
            text_summary += (
                f"--- Segment {j + 1} ---\n"
                f"Airline: {segment.airline or 'N/A'} ({segment.flight_number})\n"
                f"From: {segment.departure_airport.name} ({segment.departure_airport.id}) at {segment.departure_airport.time}\n"
                f"To: {segment.arrival_airport.name} ({segment.arrival_airport.id}) at {segment.arrival_airport.time}\n"
                f"Duration: {segment.duration} minutes\n"
                f"Class: {segment.travel_class}\n"
            )

            if flight.layovers and j < len(flight.layovers):
                layover = flight.layovers[j]
                text_summary += (
                    f"Layover at {layover.name} ({layover.id}) - {layover.duration} minutes\n"
                )

    return text_summary
@function_tool
def flight_search_2(departure_id: str, arrival_id: str, outbound_data: str, return_data: str) -> str:
    """
    Searches flights using SerpAPI's Google Flights engine.

    Args:
        departure_id (str): IATA code of departure airport.
        arrival_id (str): IATA code of arrival airport.
        outbound_data (str): Date of departure (YYYY-MM-DD).
        return_data (str): Date of return (YYYY-MM-DD).

    Returns:
        str: Formatted flight summary string.
    """
    params = {
        "api_key": api_key,
        "engine": "google_flights",
        "departure_id": departure_id,
        "arrival_id": arrival_id,
        "outbound_date": outbound_data,
        "return_date": return_data,
        "currency": "USD",
        "hl": "en",
        "deep_search": True,
    }

    try:
        response = requests.get("https://serpapi.com/search", params=params)
        response.raise_for_status()
        data = response.json()
    except requests.RequestException as e:
        return f"API request failed: {e}"
    except ValueError:
        return "Failed to parse response as JSON."

    best_flights = flight_to_string(data, flight_key='best_flights')
    other_flights = flight_to_string(data, flight_key='other_flights')

    return (
        f"<BEST FLIGHTS>\n{best_flights}\n</BEST FLIGHTS>\n"
        f"<OTHER FLIGHTS>\n{other_flights}\n</OTHER FLIGHTS>\n"
    )


f_agent = Agent(name="Flight Assistant agent",
              instructions="""
Always answer in Turkish language. Return all the given flight information

departure_id : Parameter defines the departure airport code or location kgmid. An airport code is an uppercase 3-letter code. For example, CDG is Paris Charles de Gaulle Airport and AUS is Austin-Bergstrom International Airport.

arrival_id : Parameter defines the arrival airport code or location kgmid. An airport code is an uppercase 3-letter code. You can search for it on Google Flights or IATA. For example, CDG is Paris Charles de Gaulle Airport and AUS is Austin-Bergstrom International Airport.

outbound_date : Parameter defines the outbound date. The format is YYYY-MM-DD. e.g. 2025-04-09

return_date : Parameter defines the return date. The format is YYYY-MM-DD. e.g. 2025-04-15
""",
              model="gpt-4o-mini",
              # model = 'gpt-4o',
              model_settings=ModelSettings(temperature= 0.0,
                                 max_tokens = 4096*8),
            tools= [flight_search_2,
                    hotels_search2,
                    ]
              )


h_agent = Agent(
    name = "Hotels Assistant agent",
    instructions= """Returns google hotels information as a json.     
    q : Search Query
    gl : Country""",
    model = 'gpt-4o-mini',
    tools=[hotels_search2,
           f_agent,
           ],
    model_settings=ModelSettings(temperature= 0.0,
                                 max_tokens = 4096*2),
)

In [3]:
ank = "ankara(ESB)"
ist = "istanbul(SAW)"
amst = "amsterdam(AMS)"
pekin = "Peking(PEK)"

start_point = ank 
dest_point = amst

main_question = f"""I want to travel from {start_point} to {dest_point}. 
The outbound date is 2025-05-05 and the return date will be 2025-05-11. 
What are the best hotels in Amsterdam""".strip()


main_question2 = f"""I want to travel from {start_point} to {dest_point}. 
The outbound date is 2025-05-05 and the return date will be 2025-05-11. 
tell me the cheapest travel""".strip()

main_question3 = f"""I want to travel from {start_point} to {dest_point}. 
The outbound date is 2025-05-05 and the return date will be 2025-05-11. 
tell me the cheapest travel and What are the best hotels in Amsterdam.
Calculate the cheapest option interms of flight and hotels""".strip()

In [4]:
async def process_flight(ctx: RunContextWrapper, input_data: FlightOutput):
   print(f"departure_id: {input_data.departure_id}")
   print(f"arrival_id: {input_data.arrival_id}")
   print(f"outbound_data: {input_data.outbound_data}")
   print(f"return_data: {input_data.return_data}")


async def process_hotel(ctx: RunContextWrapper, input_data: HotelOutput):
   print(f"query: {input_data.query}")
   print(f"check_in_date: {input_data.check_in_date}")
   print(f"check_out_date: {input_data.check_out_date}")   
   print(f"gl: {input_data.gl}")   

In [5]:
# query : str = Field(description=  """
# the search term should be extracted from questions. Search term can contain more than one keywords. It can be city, district, multiple destinations.
# <example>                              
# Question = I will travel to Istanbul, sariyer and like to eat sea food. 
# query = Istanbul, sariyer
# </example> 
# """)
# check_in_date : str 
# check_out_date : str 
# gl : str

In [6]:
from agents.extensions import handoff_filters

In [7]:
t_agent_output = Agent(
    name = "Travel Assistant agent",
    instructions = """ 
Investigate the users question. Determine if it is related with flight and/or hotels use flight and/or hotels tools.
If the question is not related with those generate a kind and informative answer.
If you do not have access every tool to answer the questions, tell that you do not have correspondign source
""",
model = 'gpt-4o-mini',
model_settings=ModelSettings(temperature= 0.0,
                                 max_tokens = 4096*8),
output_type=FinalModel,
handoffs=[
    handoff(
        agent=f_agent,
        on_handoff=process_flight,  
        input_type=FlightOutput,
        tool_description_override="this tool will be used to return flight info.",
        input_filter=handoff_filters.remove_all_tools,
        
    ),
    handoff(
        agent=h_agent,
        on_handoff=process_hotel, 
        input_type=HotelOutput,
        input_filter=handoff_filters.remove_all_tools,
        tool_description_override="this tool will be used to return hotels info."
    ),

]
)

In [10]:
result = Runner.run_sync(t_agent_output, main_question3)
print(result.final_output)

departure_id: ESB
arrival_id: AMS
outbound_data: 2025-05-05
return_data: 2025-05-11
### Uçuş Bilgileri

**En Uygun Uçuş:**
- **Fiyat:** $457
- **Toplam Süre:** 735 dakika
- **Gidiş:**
  - **Havayolu:** AJet (VF 3009)
  - **Kalkış:** Ankara Esenboğa Havalimanı (ESB) - 2025-05-05 21:40
  - **Varış:** Sabiha Gökçen Havalimanı (SAW) - 2025-05-05 22:50
  - **Süre:** 70 dakika
- **Aktarma:** Sabiha Gökçen Havalimanı (SAW) - 80 dakika
- **İkinci Segment:**
  - **Havayolu:** Pegasus (PC 2028)
  - **Kalkış:** Sabiha Gökçen Havalimanı (SAW) - 2025-05-06 00:10
  - **Varış:** Antalya Havalimanı (AYT) - 2025-05-06 01:30
  - **Süre:** 80 dakika
- **Aktarma:** Antalya Havalimanı (AYT) - 250 dakika
- **Üçüncü Segment:**
  - **Havayolu:** Pegasus (PC 5015)
  - **Kalkış:** Antalya Havalimanı (AYT) - 2025-05-06 05:40
  - **Varış:** Amsterdam Schiphol Havalimanı (AMS) - 2025-05-06 08:55
  - **Süre:** 255 dakika

### Otel Bilgileri

**En İyi Oteller:**

1. **via Amsterdam**
   - **Fiyat:** $94/gece
   - **

In [9]:
elma

NameError: name 'elma' is not defined

In [ ]:
import asyncio
from dataclasses import dataclass

from agents import Agent, RunContextWrapper, Runner, function_tool

@dataclass
class UserInfo:  
    name: str
    uid: int

@function_tool
async def fetch_user_age(wrapper: RunContextWrapper[UserInfo]) -> str:  
    return f"User {wrapper.context.name} is 47 years old"

async def main():
    user_info = UserInfo(name="John", uid=123)

    agent = Agent[UserInfo](  
        name="Assistant",
        tools=[fetch_user_age],
    )

    result = await Runner.run(  
        starting_agent=agent,
        input="What is the age of the user?",
        context=user_info,
    )

    print(result.final_output)  
    # The user John is 47 years old.

if __name__ == "__main__":
    asyncio.run(main())